In [ ]:
# Librerías manejo de datos
import numpy as np
import pandas as pd

# Librerías de gráficos
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de datos y ETL

Vamos a cargar un dataset sobre el peso de los niños al nacer.


In [ ]:
#birth_weight = pd.read_csv('birthweight.csv')
birth_weight = pd.read_csv('https://github.com/calabozo/orange-stats/raw/master/birthweight.csv')

In [ ]:
birth_weight.head()

In [ ]:
birth_weight["Birthweight_kg"]=birth_weight["Birthweight"]*0.453592

### Media

\\[ \mu_{x} = E[x] =  \frac{1}{N} \sum _{i=1}^N x_i\\]

Rellenar con formula y texto

In [ ]:
def mean(vector):
    pass
mean(birth_weight["Birthweight_kg"])

In [ ]:
np.mean(birth_weight["Birthweight_kg"])

### Varianza

Es la esperanza del cuadrado de la desviación de dicha variable respecto a su media. Otra definición es la media de los residuos al cuadrado.

Su formula es:

\\[ Var[x] =  \frac{1}{N} \sum _{i=1}^N \left( x_i-\bar{x} \right)^2 \\]

Pero como no sabremos $\bar{x}$ sino que siempre tendremoso que estimar la media E[x] la formula del estimador insesgado de la varianza para una muestra de la población es:

\\[ Var[x] =  \frac{1}{N-1} \sum _{i=1}^N \left( x_i-E[x] \right)^2 \\]


In [ ]:
def var(vector):
    pass
var(birth_weight["Birthweight_kg"])

In [ ]:
np.var(birth_weight["Birthweight_kg"],ddof=1)

### Mediana

La mediana es el valor de una distribución tal que la mitad de los elementos son superiores y la mitad son inferiores.

In [ ]:
def median(vector: pd.Series) -> float:
    pass
    
print(median(pd.Series([1,2,3,3.5,4,6])))
print(median(pd.Series([1,2,3,4,6])))


In [ ]:
median(birth_weight["Birthweight_kg"])

In [ ]:
np.median(birth_weight["Birthweight_kg"])

### Cuantiles

Los cuantiles suelen usarse por grupos que dividen la distribución en partes iguales; entendidas estas como intervalos que comprenden la misma proporción de valores. Los más usados son:

* Los cuartiles, que dividen a la distribución en cuatro partes (corresponden a los cuantiles 0,25; 0,50 y 0,75);
* Los quintiles, que dividen a la distribución en cinco partes (corresponden a los cuantiles 0,20; 0,40; 0,60 y 0,80);
* Los deciles, que dividen a la distribución en diez partes;
* Los percentiles, que dividen a la distribución en cien partes.


In [ ]:
np.quantile(birth_weight["Birthweight_kg"],[0.25,0.5,0.75])

### Boxplot

In [ ]:
sns.boxplot(x=birth_weight["Birthweight_kg"])

### Histograma

In [ ]:
sns.histplot(birth_weight["Birthweight_kg"], binwidth=0.5,label="0.5 binwidth")
sns.histplot(birth_weight["Birthweight_kg"], bins=20,color="red",label="20 bins")
plt.xlabel("Weight [kg]")
plt.legend()

### Curva de densidad (kernel density estimation)

In [ ]:
sns.kdeplot(birth_weight["Birthweight_kg"], bw_adjust=0.5,label="bw_adjust=0.5")
sns.kdeplot(birth_weight["Birthweight_kg"], bw_adjust=2,color="red",label="bw_adjust=1")
plt.xlabel("Weight [kg]")
plt.legend()

### Violin plot

In [ ]:
sns.violinplot(data=birth_weight,x="Birthweight_kg")

# Tests estadísticos

¿Es la media de estas dos poblaciones iguales?

In [ ]:
weight_NO_smoke = birth_weight.loc[birth_weight["smoker"]==0,"Birthweight_kg"]
weight_YES_smoke = birth_weight.loc[birth_weight["smoker"]==1,"Birthweight_kg"]

In [ ]:
weight_NO_smoke.describe()

In [ ]:
weight_YES_smoke.describe()

In [ ]:
sns.kdeplot(weight_NO_smoke,label="No fumadora")
sns.kdeplot(weight_YES_smoke,color="red",label="Fumadora")
plt.xlabel("Weight [kg]")
plt.legend()

In [ ]:
diff_NO_YES=np.mean(weight_NO_smoke.values)-np.mean(weight_YES_smoke.values)
diff_YES_NO=np.mean(weight_YES_smoke.values)-np.mean(weight_NO_smoke.values)
print(f"La diferencia de medias NO_smoker-YES_smoker es:{np.round(diff_NO_YES,2)}")
print(f"La diferencia de medias YES_smoker-NO_smoker es:{np.round(diff_YES_NO,2)}")

¿Cómo de probable es que estos cambios hayan ocurrido al azar?

In [ ]:

X1=weight_NO_smoke
X2=weight_YES_smoke

def perm_func(X1: pd.Series, X2:pd.Series) -> (pd.Series,pd.Series): 
    X = X1.append(X2).reset_index(drop=True)

    X1_reorder=X.sample(frac=0.5)
    X2_reorder=X.drop(X1_reorder.index)
    return (X1_reorder.reset_index(drop=True),X2_reorder.reset_index(drop=True))

X1_reorder, X2_reorder= perm_func(weight_NO_smoke,weight_YES_smoke)

In [ ]:
mean_values = []
num_mean_X1_larger_X2 = 0
num_mean_X2_larger_X1 = 0

total_trials = 10000
for i in range(total_trials):
    X1_reorder, X2_reorder= perm_func(weight_NO_smoke,weight_YES_smoke)
    
    mean_diff = np.mean(X1_reorder.values) - np.mean(X2_reorder.values)
    mean_values.append(mean_diff)
    
    if mean_diff>diff_NO_YES:
        num_mean_X1_larger_X2 += 1
    
    if mean_diff<diff_YES_NO:
        num_mean_X2_larger_X1 += 1
    


In [ ]:
print(f"""La probabilidad por puro azar la media de X1 menos X2 sea mayor que la diferencia original: 
    {np.round(num_mean_X1_larger_X2/total_trials*100,2)}%""")

print(f"""La probabilidad por puro azar la media de X1 menos X2 sea menor que la diferencia original: 
    {np.round(num_mean_X2_larger_X1/total_trials*100,2)}%""")

print(f"""La probabilidad de que por puro azar los valores de NO_smoker y YES_smoker se hayan reordenado
para dar diferencias tan grandes es:{np.round((num_mean_X2_larger_X1+num_mean_X1_larger_X2)/total_trials*100,2)}%""")

In [ ]:
from scipy.stats import norm

#sns.displot(mean_values)
sns.kdeplot(mean_values)
plt.axvline(diff_NO_YES, 0,2,color="#FF0000")
plt.axvline(diff_YES_NO, 0,2,color="#FF0000")
x=np.linspace(-1, 1, 100)
plt.plot( x, norm.pdf(x, 0, np.std(mean_values,ddof=1)))


In [ ]:
np.var(mean_values)

Vemos que la probabilidad de que esos datos se hayan conseguido por puro hazar es muy baja, inferior al 5%-

### T-student

El t-test es usado cuando tienes dos grupos de datos y quieres saber si la media de ambos grupos es igual o no. Tenemos dos hipótesis:
\\[H_0 : \mu(X) = \mu(Y) \\]
\\[H_1 : \mu(X) \ne \mu(Y) \\]

Empezamos con las siguientes suposiciones:
1. Las muestras han sido seleccionas por muestreo aleatorio simple dentro de la población.
2. Las poblaciones siguen una distribución normal. (sino usar test Wilcoxon-Mann-Whitney)
3. Ambas poblaciones tienen la misma varianza. (sino usar test Welch)
4. Las poblaciones no están correladas. (sino usar t-test emparejado)

Primero calculamos la media y varianza de ambos grupos. El test para verificar si la hipótesis nula $H_0$ es cierta puede ser calculado como sigue:

\\[ t=\frac{E[X]-E[Y]}{s_p·\sqrt{\frac{1}{n_x}+\frac{1}{n_y}}} \\]

Donde $s_p$ es la desviación estandar compuesta, calculada como:

\\[ s^2_p=\frac{(n_x-1)Var[X]+(n_y-1)Var[Y]}{n_x+n_y-2} \\]

Donde $n_x$ y $n_y$ son los números de muestras en cada uno de los grupos muestreados. El número de grados de libertad es 
$d.f.=n_x+n_y-2$. Podemos asumir que las dos varianzas son iguales si ambas pasan el test de varianza de Fisher(F-test).


In [ ]:
from scipy import stats
t, p = stats.ttest_ind(weight_NO_smoke,weight_YES_smoke)
p


# Ejemplo practico

¿Cuantos caramelos hay en estos grupos?

![](https://github.com/calabozo/orange-stats/raw/master/caramelos.jpg)

# Regresiones lineales

Modelo matemático usado para aproximar la relación de dependencia entre una variable dependiente $Y$, diferentes variables independientes definidas en la matriz $X$ y un término aleatorio $\varepsilon$. Este modelo puede ser expresado como:

\\[
\hat{Y}=\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p = \sum \beta_k X_k
\\]


Tratamos de encontrar la predicción $\hat{Y}$ tal que minimize el error cuadrático medio:
\\[
MSE = {1 \over n} \sum_{i=0}^n{(Y-\hat{Y})^2}
\\]



![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/400px-Linear_regression.svg.png)


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(birth_weight,test_size=0.33, random_state=42)

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
train.head()

In [ ]:
model = ols(formula='Birthweight_kg ~ mheight + motherage + fheight + fage + mnocig', data=birth_weight)
res = model.fit()
print(res.summary())

In [ ]:
model = ols(formula='Birthweight_kg ~ mheight+ mnocig', data=birth_weight)
res = model.fit()
print(res.summary())

## Clasificación: Regresión logística

En lugar de realizar una predicción de un valor queremos hacer un clasificador.
Podemos tratar de asignar una probabilidad. Pero hay un problema porque la regresión lineal va entre 0 y 1.

Para ello transformarmos la regresión lineal mediante la función logit:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Logit.svg/350px-Logit.svg.png)

#### función de enlace (link function)

Para pasar del dominio de números reales $(-\infty,\infty)$ al de probabilidades $[0,1]$ a vamos a utilizar la **función logística**:
\\[
p = h(x)=  \frac{1}{1+e^{-x}}
\\]
Su inversa se conoce como la función **logit**:
\\[
h^{-1}(p) = log \left( \frac{p}{1-p} \right)
\\]

Es decir, cuando estemos trabajando con una **distribución binomial** un modelo lineal del tipo:
\\[
y = \beta \vec{x}+\beta_0
\\]
lo podemos trasnformar en:
\\[
y = p(x) = \frac{1}{1+e^{-\beta \vec{x}-\beta_0}} 
\\]
Ahora $p(x)$ es una función que muestra valores en el rango $[0,1]$, puede ser considerada como una probabilidad.

Y definiendo un umbral podríamos tener el siguiente clasificador:
* Seleccionamos clase 1 si p(x)>=0.5
* Seleccionamos clase 0 si p(x)< 0.5



Es decir, tenemos una probabilidad, su valor está en el rango $[0,1]$:
\\[
    p = \frac{1}{1-e^{-\hat{Y}}}= \frac{1}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}   
\\]

Definimos la razón de monomios (Odds ratio) como el cociente entre dos probabilidades, su valor está en el rango $[0,\infty]$:

\\[
 Odds = \frac{p}{1-p}=\frac{\frac{1}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}}{\frac{e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}}=e^{(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}
\\]

Si aplicamos el logaritmo a la razón de monomios tenemos un valor que está en el rango $[-\infty,\infty]$:
\\[
 log(Odds)= log \left(\frac{p}{1-p} \right) = \beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p
\\]

La función de coste que vamos a tratar de minimizar será:
\\[
\begin{split}
Cost(p(x),y) &= {1 \over n} \sum_{i=0}^n{(y-\hat{y})^2}\\
Cost(p(x),y) &= {1 \over n} \sum_{i=0}^n{(y-p(x))^2}
\end{split}
\\]
Que transformaremos en:
\\[
Cost(p(x),y) = -y ·log(p(x))-(1-y)·log(1-p(x))
\\]


### Ejemplo:Churn rate

Vamos a utilizar un dataset publicado por IBM en [kaggle](https://www.kaggle.com/blastchar/telco-customer-churn).


En este ejemplo vamos a cargar el dataset proporcionado y ver si somos capaces de ver qué usuarios son los que corren más riesgo de irse.

El conjunto de datos incluye información sobre:

* Clientes que se fueron en el último mes: la columna se llama Churn
* Servicios para los que se ha registrado cada cliente: teléfono, líneas múltiples, Internet, seguridad en línea, copia de seguridad en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas
* Información de la cuenta del cliente: cuánto tiempo han sido cliente (columna tenure), contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales
* Información demográfica sobre los clientes: sexo, rango de edad y si tienen socios y dependientes

In [ ]:
#df_churn=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df_churn=pd.read_csv("https://github.com/calabozo/orange-stats/raw/master/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df_churn.head()

In [ ]:
df_churn.dtypes

In [ ]:
df_churn=df_churn.drop(
    columns=["customerID","OnlineSecurity","OnlineBackup",
             "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","TotalCharges"])

In [ ]:
df_churn.dtypes

In [ ]:
from statsmodels.formula.api import logit
# https://www.statsmodels.org/stable/generated/statsmodels.formula.api.logit.html

In [ ]:
df_churn["Churn"].unique()

In [ ]:
df_churn["Churn"]=df_churn["Churn"].astype('category')
df_churn["Churn_id"]=df_churn["Churn"].cat.codes

In [ ]:
from sklearn.model_selection import train_test_split
df_churn_train,df_churn_test = train_test_split(df_churn,test_size=0.33, random_state=42)
df_churn_test=df_churn_test.copy()

In [ ]:
churn_model=logit(formula="Churn_id ~ tenure+C(InternetService)+MonthlyCharges+C(PaymentMethod)",
                  data=df_churn_train).fit()
churn_model.summary()

#### Salida de un modelo

La salida de esta predicción es el logaritmo de la razón de monomios:
\\[
 log(Odds)= log \left(\frac{p}{1-p} \right) = \beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p
\\]

In [ ]:
user_test = df_churn_test.iloc[5:6,:]

In [ ]:
log_odds=churn_model.predict(user_test, linear=True).values[0]

In [ ]:
print(f"Es {np.round(np.exp(log_odds),2)} veces más probable que user_test haga Churn que permanezca en la compañia")

In [ ]:
prob_est = churn_model.predict(user_test, linear=False).values[0]

In [ ]:
print(f"""Aunque no se corresponde con la probabilidad teórica, 
una aproximación a la probabilidad de churn para el usuario test rondaría {np.round(prob_est,2)}""")

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

prob_est == sigmoid(log_odds)

#### Medidas de calidad de un modelo

In [ ]:
cnf_mat=pd.crosstab(index=df_churn_test["Churn"],columns=churn_model.predict(df_churn_test)>0.5 )

In [ ]:
df_churn_test["pred"]=churn_model.predict(df_churn_test)>0.5
df_churn_test["pred"] = df_churn_test["pred"].replace({True: 'Yes', False: 'No'})
df_churn_test=df_churn_test.dropna(subset=["Churn"])


In [ ]:
from sklearn.metrics import confusion_matrix
cnf_mat=confusion_matrix(df_churn_test["Churn"], df_churn_test["pred"], labels=["No","Yes"] )
cnf_mat

In [ ]:
FP = cnf_mat.sum(axis=0) - np.diag(cnf_mat) 
FN = cnf_mat.sum(axis=1) - np.diag(cnf_mat)
TP = np.diag(cnf_mat)
TN = cnf_mat.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print(f"Recall o True Positive Rate: {TPR}")
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print(f"True Negative rate: {TPR}")
# Precision or positive predictive value
PPV = TP/(TP+FP)
print(f"Precision: {PPV}")
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(f"Accuracy: {ACC}")

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(df_churn_test["Churn"]=="Yes", churn_model.predict(df_churn_test))
plt.plot(fpr, tpr)
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

roc_auc_score(df_churn_test["Churn"], churn_model.predict(df_churn_test))

#### Mejorando la calidad del modelo

In [ ]:
df_payment=pd.crosstab(index=df_churn_train["Churn"],columns=df_churn_train["PaymentMethod"])
df_payment

In [ ]:
from scipy import stats
from statsmodels.stats.proportion import proportion_confint

In [ ]:
proportion_confint(3, nobs=15, method='binom_test')

In [ ]:
df_payment.apply(lambda x:proportion_confint(x[0],nobs=x[0]+x[1],method='binom_test'))

In [ ]:
df_churn["ElectronicCheck"]=df_churn.PaymentMethod=="Electronic check"

In [ ]:
df_churn.head()

In [ ]:
df_churn_train,df_churn_test = train_test_split(df_churn,test_size=0.33, random_state=42)

In [ ]:
churn_new_model=logit(formula="Churn_id ~ tenure+C(InternetService)+C(PaperlessBilling)+C(Contract)+MonthlyCharges+C(ElectronicCheck)+C(SeniorCitizen)",
                  data=df_churn_train).fit()
churn_new_model.summary()

In [ ]:
fpr_new, tpr_new, thresholds_new = roc_curve(df_churn_test["Churn"]=="Yes", churn_new_model.predict(df_churn_test))
plt.plot(fpr, tpr)
plt.plot(fpr_new, tpr_new)